# Designing a Conjoint Survey

Selamat datang di hari kedua pada seri webinar *Conjoint Analysis for Retail Business*.

Sebagai ikhtisar, kita akan membahas beberapa topik seputar:
* implementasi analisis konjoin
* identifikasi atribut dan level
* tips melakukan analisis konjoin

## Studi Konjoin

Dari beberapap jenis studi konjoin, **rating-based** dan **choice-based** adalah dua pendekatan yang paling sering digunakan. Tergantung pendekatan mana yang dipakai, secara umum, diagram berikut mengilustrasikan langkah-langkah dalam sebuah studi konjoin.

```{figure} ../../assets/steps-in-conjoint-study.png
:name: steps-in-conjoint-study
Langkah-langkah dalam studi konjoin ({cite:t}`applied_conjoint_analysis`).
```

<!-- TODO: tambahkan identifikasi masalah di diagram -->

Berdasarkan diagram di atas, langkah pertama sebelum memulai sebuah studi konjoin tentu saja adalah **menentukan studi konjoin**. Kali ini, kita akan melakukan studi konjoin pada bidang retail, khususnya pada produk mobil.


```{warning}
Dalam praktiknya, setelah mendefinisikan masalah dan bagaimana hasil dari studi akan digunakan, masalah berikutnya **tidak hanya menentukan atribut-atribut dari sebuah produk**, tapi juga menentukan **bagaimana produk dengan atribut tersebut dapat dijual atau bahkan memotong penjualan produk lama.
```

### Identifikasi Atribut dan Level

Seperti yang disebutkan pada materi hari pertama, pendekatan studi konjoin bertujuan untuk menemukan "fungsi preferensi" dari sebuah produk berdasarkan atributnya. Oleh karena itu, identifikasi atribut menjadi **langkah yang paling penting** dalam sebuah studi konjoin.

Atribut dari sebuah produk dapat dibagi menjadi 2 jenis:
* kategoris

    Contoh atribut kategoris seperti nama brand, jenis transmisi pada mobil, dan sejenisnya. Sehingga, level dari atribut ini juga berbentuk kategoris, tapi tidak harus memiliki urutan yang jelas.

* kuantitatif

    Berbeda dengan atribut kategoris, atribut kuantitatif biasanya berupa sebuah interval atau rasio.

````{dropdown} Kategoris vs Kuantitatif
Dari gambar berikut ini, coba tentukan mana yang bisa menjadi atribut kategoris dan mana yang kuantitatif?

```{image} ../../assets/product-attributes-iphone-attention.png
:align: center
```
````

Pada dasarnya, pemilihan atribut dan level sebenarnya lebih ke urusan kreativitas dibanding sains ({cite:t}`applied_conjoint_analysis`). Lalu, darimana kita bisa menentukan atribut-atribut tersebut?

* informasi yang dikumpulkan dari survei konsumen yang pernah dilakukan sebelumnya
* sumber eksternal seperti [Consumer Insights dari Google](https://www.thinkwithgoogle.com/intl/en-apac/consumer-insights/) atau sejenisnya, yang relevan dengan produk kita
* sesi workshop atau brainstorming internal perusahaan
* melakukan analisis terhadap _revealed preference data_


### Membuat Alternatif Survei dengan Python

Misal kita ingin mendesain survei konjoin untuk mobil sport dengan pilihan atribut dan masing-masing level berikut:
* `seat`: 2, 4, 5
* `convert`: yes, no
* `trans`: manual, auto
* `segment`: basic, fun, racer
* `price`: 30, 40, 50

Salah satu caranya adalah dengan membuat semua kombinasi yang mungkin dari masing-masing atribut dan juga level tersebut

In [ ]:
import pandas as pd

from itertools import product

In [ ]:
att_seat = [2, 4, 5]
att_convert = ["Yes", "No"]
att_trans = ["manual", "auto"]
att_segment = ["basic", "fun", "racer"]
att_price = [30, 40, 50]

In [ ]:
all_combinations = product(att_seat, att_convert, att_trans, att_segment, att_price)

attributes = pd.DataFrame(
    [combination for combination in all_combinations],
    columns=["seat", "convert", "trans", "segment", "price"]
)

In [ ]:
attributes

In [ ]:
for i in range(100):
    display(attributes.sample(3))
    break

In [ ]:
surveys = []
n_alternatives = 3
n_questions = 10

for i in range(n_questions):
    sample = attributes.sample(n_alternatives)
    sample["questions"] = i + 1
    sample["alt"] = list(range(1, n_alternatives+1))
    surveys.append(sample)

df_surveys = pd.concat(surveys).reset_index(drop=True)
df_surveys

Metode pembuatan profil seperti di atas disebut juga sebagai **full factorial design**.

Metode ini kurang begitu praktis mengingat jumlah profil yang dihasilkan akan semakin membesar dengan semakin banyaknya pilihan atribut dan level masing-masing.

In [ ]:
len(att_seat) * len(att_convert) * len(att_price) * len(att_segment) * len(att_trans)

Oleh karena itu, daripada memuat semua **108** profil, kita bisa menggunakan pendekatan **fractional factorial design**. Dengan pendekatan ini, kita bisa memiih _fraction_ dari total profil yang mungkin.

Misalkan dari 108 profil yang mungkin, kita bisa memilih setengahnya, yaitu 54 untuk dipilih secara "sistematis" berdasarkan beberapa faktor seperti cost, kombinasi atribut-level yang masuk akal, durasi interview, dan lainnya.

```{admonition} Contoh fractional factorial design
Dari 4 atribut beserta level-levelnya:
* price: USD 550, USD 700, USD 850
* face ID: with face ID, without face ID
* battery: 6 hours, 10 hours
* bonus storage: "standard", "plus"

jumlah keseluruhan profil yang mungkin dibentuk adalah $3 \times 2 \times 2 \times 2 = 24$. Tapi, kita memutuskan untuk menggunakan 12 profil (setengahnya) dengan urutan acak berikut.
```

In [ ]:
att_price = [550, 700, 850]
att_faceid = ["Yes", "No"]
att_battery_hours = [6, 10]
att_storage = ["standard", "plus"]

all_combinations = product(att_price, att_faceid, att_battery_hours, att_storage)

attributes = pd.DataFrame(
    [combination for combination in all_combinations],
    columns=["Price (USD)", "Face ID", "Battery", "Bonus Storage"]
)
attributes.sample(12, random_state=1).reset_index(drop=True)

## Conjoint Data Collection

Kita bisa mengumpulkan data konjoin dari mana saja, tidak hanya melalui survei langsung. Jika kita sudah mempunyai _base user_ dan data transaksi yang cukup banyak, kita bisa merekrontuksi data tersebut menjadi data konjoin.

Misalkan, untuk membuat data konjoin dari sebuah data transaksi di sebuah supermarket dengan cara melihat **apa yang pembeli beli** dan **apa yang tersedia** saat itu.

```{figure} ../../assets/groceries-meme.jpg
:width: 240px
:name: groceries-meme
Kemarin kayaknya ada deh? ([ref](https://knowyourmeme.com/photos/1608093-ryan-gosling))
```

```{note}
Data yang direkontruksi dari proses tersebut disebut juga sebagai **_revealed preference data_**.
```

Akan tetapi, kadang kita tidak bisa menemukan yang kita cari dari data histori transaksi. Terutama, ketika kita mencoba menawarkan fitur baru di produk kita, yang tentu saja belum tersedia (apalagi dibeli).

Dalam kasus ini, kita bisa membuat sebuah **survei** kepada calon pembeli untuk membuat **"pilihan"**. Data yang dihasilkan dari proses inilah yang dinamakan dengan **_conjoint data_**.

```{note}
Nama lain dari data yang dihasilkan dari survei tersebut adalah **_stated preference data_**.
```

```{figure} ../../assets/stated-preference-data.png
:name: stated-preference-data
Contoh survei kepada konsumen untuk memilih mobil mana yang kiranya akan dipilih atau dibeli ({cite:t}`datacamp_choice_modeling`)
```

### Full Profile

Seperti yang sudah disebutkan di materi hari pertama, pada _full-profile_, kita akan menggunakan semua atribut untuk ditampilkan kepada responden. Akan tetapi, jika jumlah atribut (dan level masing-masing) tidak terlalu besar, kita bisa menampilkan semua kombinasi (profil).


### Trade-Off Matrix

Responden akan diminta untuk menilai alternatif produk dengan kombinasi **2 atribut** sekaligus. Atribut-atribut tersebut akan berubah sedemikian hingga responden akan menilai semua profil dari 2 atribut tersebut. Perhatikan contoh berikut.

```{figure} ../../assets/trade-off-matrix-example.png
:name: trade-off-matrix-example
Contoh hasil survei menggunakan _trade-off matrix_. Responden ini cenderung memilih `Goodyear` yang bertahan hingga 50k mil dan tetap memilih brand yang sama dengan daya tahan yang lebih rendah.
```

```{warning}
Meskipun metode ini sederhana, kita tidak tahu asumsi apa yang dibuat oleh responden untuk atribut-atribut lain yang tidak dimunculkan.
```

### Contoh _Conjoint Data_

bagaimana pun kita mengumpulkan data konjoin, dan tergantung _tool_ yang kita gunakan untuk mengumpulkan data, bentuk data konjoin memiliki format **memanjang** ataupun **melebar**.

In [ ]:
sportscar_long = pd.read_csv("../../data/sportscar_choice/sportscar_choice_long.csv")
sportscar_wide = pd.read_csv("../../data/sportscar_choice/sportscar_choice_wide.csv")

In [ ]:
sportscar_long

In [ ]:
sportscar_wide

Data dengan format melebar ini biasanya kita temukan jika kita menggunakan tools seperti Google Form atau Qualtrics. Untuk itu, kita perlu merubah data dengan format melebar ini menjadi format memanjang untuk memudahkan kita dalam melakukan eksplorasi dan visualisasi.

In [ ]:
sportscar_long

In [ ]:
sportscar_wide["id"] = sportscar_wide.index
sportscar = pd.wide_to_long(
    sportscar_wide,
    ["seat", "trans", "convert", "price"],
    i="id", j="alt", sep="."
).reset_index(level="alt")
sportscar

In [ ]:
fields = [
    "resp_id", "ques", "alt", "segment", "seat",
    "trans", "convert", "price", "choice"
]
sportscar = sportscar.sort_values(["resp_id", "ques", "alt"])[fields].reset_index(drop=True)
sportscar

In [ ]:
sportscar["choice"] = (sportscar["choice"] == sportscar["alt"]).astype("int")
sportscar

In [ ]:
(sportscar_long == sportscar).sum()

## Tips dalam Melakukan Studi Konjoin

Perlu diingat bahwa **atribut** merupakan "dimensi" produk, seperti harga, warna, ukuran, merk, lokasi, dan lainnya. Kaitannya dengan atribut ({cite:t}`applied_conjoint_analysis`):
* Masukkan ke dalam pertanyaan survei, atribut-atribut yang menurut kamu penting bagi konsumen ketika mereka hendak membeli dan/atau atribut lainnya yang ingin kamu pastikan dampaknya. Misalkan, kamu memahami bahwa preferensi konsumen tergantung kepada harga dan ukuran produk, dan kamu ingin memastikan apakah pilihan warna juga penting, maka kamu bisa sertakan ketiga atribut tersebut, yaitu harga, ukuran, dan juga warna
* Usahakan **jangan** memasukkan **lebih dari 7 atribut** supaya tidak membuat konsumen semakin bingung terlihat terlalu memaksa.
* Secara umum, **harga** menjadi satu atribut yang sulit untuk tidak disertakan, jadi sertakan saja.

Selain atribut, kita juga perlu memperhatikan soal **level**. Level merupakan opsi nilai yang mungkin dari sebuah atribut. Misalkan, atribut warna mungkin memiliki 3 level, yaitu merah, kuning, dan hijau.
* Pastikan bahwa setidaknya kamu memiliki **minimal 2 level per atribut**.
* Level harus **detail**, misalkan level dari atribut resolusi kamera sebuah handphone adalah 46 MP, 50 MP, atau 52 MP. Bukan "lebih dari 45 MP" ataupun "tidak lebih dari 50 MP".
* Pastikan bahwa level setiap atribut **realistis** (tapi **belum perlu _feasible_** di waktu studi dilakukan)
* Level-level pada satu atribut harus saling lepas satu sama lain. Misalkan untuk produk _snack_ untuk atribut rasa, hindari nilai seperti "manis" dan "pedas manis". Sebagai alternatif, kita bisa buat rasa seperti "caramel" dan "spicy".

Selain itu, untuk atribut dan level-nya:
* Pastikan untuk menggunakan bahasa atau istilah yang mudah dimengerti oleh konsumen.
* Jika kesulitan, **_a picture is worth a thousand words_**.